# Dogs vs Cat Redux

In this tutorial, you will learn how generate and submit predictions to a Kaggle competiton

[Dogs vs. Cats Redux: Kernels Edition](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)
    
    

In [1]:
from datetime import datetime

In [2]:
#Verify we are in the lesson1 directory
%pwd

u'/home/ubuntu/fastai/fastai_bmt_reworking/dogsCats'

In [3]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
#DATA_HOME_DIR = current_dir+'/data/redux'

In [4]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [5]:
HOMEPATH = "/home/ubuntu/fastai/"
DATA_HOME_DIR = HOMEPATH + "data/dogscats/"

In [6]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
#import modules
from utils import *
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

('os.getcwd:', '/home/ubuntu/fastai')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


## Finetuning and Training

In [7]:
DATA_HOME_DIR

'/home/ubuntu/fastai/data/dogscats/'

In [8]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR                    #use this when all code tested and ready for longer run
#path = DATA_HOME_DIR + 'sample/'         #use this during development - smaller dataset
test_path = DATA_HOME_DIR + 'test/'      #We use all the test data
results_path=DATA_HOME_DIR + 'results/'  #results always in this directory.
train_path=path + 'train/'
valid_path=path + 'valid/'
print "path:", path
print "test_path:", test_path
print "results_path:", results_path
print "train_path:", train_path
print "valid_path:", valid_path


/home/ubuntu/fastai/data/dogscats
path: /home/ubuntu/fastai/data/dogscats/
test_path: /home/ubuntu/fastai/data/dogscats/test/
results_path: /home/ubuntu/fastai/data/dogscats/results/
train_path: /home/ubuntu/fastai/data/dogscats/train/
valid_path: /home/ubuntu/fastai/data/dogscats/valid/


In [9]:
#import Vgg16 helper class
vgg = Vgg16()

In [10]:
print type(vgg)

<type 'instance'>


In [14]:
print (type(vgg.model), type(vgg.model.layers), len(vgg.model.layers))
print (vgg.model.summary())

(<class 'keras.models.Sequential'>, <type 'list'>, 38)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________

In [28]:
startTime= datetime.now()
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))
print ("batches:", type(batches))
print ("val_batches:", type(val_batches))

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Time elpased (hh:mm:ss.ms) 0:00:00.507923
('batches:', <class 'keras.preprocessing.image.DirectoryIterator'>)
('val_batches:', <class 'keras.preprocessing.image.DirectoryIterator'>)


In [19]:
print (type(batches))
print (type(val_batches))

<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'keras.preprocessing.image.DirectoryIterator'>


In [22]:
val_classes = val_batches.classes
trn_classes = batches.classes
print ("val_classes:", type(val_classes), val_classes.shape)
print("val_classes[0:10]:", val_classes[0:10])
print ("trn_classes:", type(trn_classes), trn_classes.shape)
print("trn_classes[0:10]:", trn_classes[0:10])
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)
print ("val_labels:", val_labels[0])
print ("trn_labels:", trn_labels[0])
print ("trn_labels.shape:", trn_labels.shape)

('val_classes:', <type 'numpy.ndarray'>, (2000,))
('val_classes[0:10]:', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('trn_classes:', <type 'numpy.ndarray'>, (23000,))
('trn_classes[0:10]:', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32))
('val_labels:', array([ 1.,  0.]))
('trn_labels:', array([ 1.,  0.]))
('trn_labels.shape:', (23000, 2))


In [23]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=3

In [24]:
print ("before pop, # of layers:", len(vgg.model.layers))
vgg.model.pop()
print ("after pop, # of layers:", len(vgg.model.layers))
for layer in vgg.model.layers: 
    layer.trainable=False

('before pop, # of layers:', 38)
('after pop, # of layers:', 37)


In [25]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [27]:
vgg.model.add(Dense(2, activation='softmax'))
print ("after adding dense layer, # of layers:", len(vgg.model.layers))


('after adding dense layer, # of layers:', 38)


In [29]:
startTime= datetime.now()
vgg.finetune(batches)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.01

Time elpased (hh:mm:ss.ms) 0:00:00.036943


In [ ]:
print "no_of_epochs:", no_of_epochs
print os.getcwd()
print "results_path:", results_path
DIR = results_path
print "# files in ", DIR, len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])


In [ ]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
startTime= datetime.now()
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    #latest_weights_filename = 'ft%d.h5' % epoch
    #vgg.model.save_weights(results_path+latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

#sample data takes approx 16s/epoch

In [ ]:
DIR = results_path
print "# files in ", DIR, len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])


## Generate Predictions

Let's use our new model to make predictions on the test dataset

In [ ]:
print "test_path:", test_path

In [ ]:
startTime= datetime.now()
batches, preds = vgg.test(test_path, batch_size = batch_size*2)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [ ]:
print "batches:", type(batches)
print "preds:", type(preds), preds.shape

In [ ]:
#For every image, vgg.test() generates two probabilities 
#based on how we've ordered the cats/dogs directories.
#It looks like column one is cats and column two is dogs
print preds[:5]

filenames = batches.filenames
print filenames[:5]

In [ ]:
#You can verify the column ordering by viewing some images
from PIL import Image
print filenames[0]
Image.open(test_path + filenames[0])

In [ ]:
print filenames[1]
Image.open(test_path + filenames[1])

In [ ]:
print filenames[2]
Image.open(test_path + filenames[2])

In [ ]:
for i in range(0,preds.shape[0]):
    if preds[i,0]<0.1:
        print "at i=", i, preds[i]
    #print i, preds[i], preds[i,0]
    if i >= 200: break
print "end"

In [ ]:
#Save our test results arrays so we can use them again later
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

## Validate Predictions

Keras' *fit()* function conveniently shows us the value of the loss function, and the accuracy, after every epoch ("*epoch*" refers to one full run through all training examples). The most important metrics for us to look at are for the validation set, since we want to check for over-fitting. 

- **Tip**: with our first model we should try to overfit before we start worrying about how to reduce over-fitting - there's no point even thinking about regularization, data augmentation, etc if you're still under-fitting! (We'll be looking at these techniques shortly).

As well as looking at the overall metrics, it's also a good idea to look at examples of each of:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with highest probability that are correct)
4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
5. The most uncertain labels (ie those with probability closest to 0.5).

Let's see what we can learn from these examples. (In general, this is a particularly useful technique for debugging problems in the model. However, since this model is so simple, there may not be too much to learn at this stage.)

Calculate predictions on validation set, so we can find correct and incorrect examples:

In [ ]:
vgg.model.load_weights(results_path+latest_weights_filename)

In [ ]:
print "valid_path:", valid_path

In [ ]:
val_batches, probs = vgg.test(valid_path, batch_size = batch_size)

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

#Round our predictions to 0/1 to generate labels
our_predictions = probs[:,0]
our_labels = np.round(1-our_predictions)

In [ ]:
from keras.preprocessing import image

#Helper function to plot images by index in the validation set 
#Plots is a helper function in utils.py
def plots_idx(idx, titles=None):
    plots([image.load_img(valid_path + filenames[i]) for i in idx], titles=titles)
    
#Number of images to view for each visualization task
n_view = 4

In [ ]:
print type(our_labels), our_labels.shape
print type(expected_labels), expected_labels.shape

In [ ]:
#1. A few correct labels at random
correct = np.where(our_labels==expected_labels)[0]
print "Found %d correct labels" % len(correct)
idx = permutation(correct)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
#2. A few incorrect labels at random
incorrect = np.where(our_labels!=expected_labels)[0]
print "Found %d incorrect labels" % len(incorrect)
idx = permutation(incorrect)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
#3a. The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print "Found %d confident correct cats labels" % len(correct_cats)
if len(correct_cats)!=0:
    most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
    plots_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])
else:
    print "zero confident cats. SOMETHING WRONG!!!"

In [ ]:
#3b. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print "Found %d confident correct dogs labels" % len(correct_dogs)
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])

In [ ]:
#4a. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print "Found %d incorrect cats" % len(incorrect_cats)
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_view]
    plots_idx(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#4b. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print "Found %d incorrect dogs" % len(incorrect_dogs)
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:n_view]
    plots_idx(incorrect_dogs[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain])

Perhaps the most common way to analyze the result of a classification model is to use a [confusion matrix](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). Scikit-learn has a convenient function we can use for this purpose:

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)

We can just print out the confusion matrix, or we can show a graphical view (which is mainly useful for dependents with a larger number of categories).

In [ ]:
plot_confusion_matrix(cm, val_batches.class_indices)

## Submit Predictions to Kaggle!

Here's the format Kaggle requires for new submissions:
```
imageId,isDog
1242, .3984
3947, .1000
4539, .9082
2345, .0000
```

Kaggle wants the imageId followed by the probability of the image being a dog. Kaggle uses a metric called [Log Loss](http://wiki.fast.ai/index.php/Log_Loss) to evaluate your submission.

In [ ]:
#Load our test predictions from file
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
#Grab the dog prediction column
isdog = preds[:,1]
print "Raw Predictions: " + str(isdog[:5])
print "Mid Predictions: " + str(isdog[(isdog < .6) & (isdog > .4)])
print "Edge Predictions: " + str(isdog[(isdog == 1) | (isdog == 0)])

[Log Loss](http://wiki.fast.ai/index.php/Log_Loss) doesn't support probability values of 0 or 1--they are undefined (and we have many). Fortunately, Kaggle helps us by offsetting our 0s and 1s by a very small value. So if we upload our submission now we will have lots of .99999999 and .000000001 values. This seems good, right?

Not so. There is an additional twist due to how log loss is calculated--log loss rewards predictions that are confident and correct (p=.9999,label=1), but it punishes predictions that are confident and wrong far more (p=.0001,label=1). See visualization below.

In [ ]:
#Visualize Log Loss when True value = 1
#y-axis is log loss, x-axis is probabilty that label = 1
#As you can see Log Loss increases rapidly as we approach 0
#But increases slowly as our predicted probability gets closer to 1

#this keeps throwing error. find fix later.
'''
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss

x = [i*.0001 for i in range(1,10000)]
y = [log_loss([1],[[i*.0001,1-(i*.0001)]],eps=1e-15) for i in range(1,10000,1)]

plt.plot(x, y)
plt.axis([-.05, 1.1, -.8, 10])
plt.title("Log Loss when true label = 1")
plt.xlabel("predicted probability")
plt.ylabel("log loss")

plt.show()
'''

In [ ]:
#So to play it safe, we use a sneaky trick to round down our edge predictions
#Swap all ones with .95 and all zeros with .05
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

Here we join the two columns into an array of [imageId, isDog]

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
DATA_HOME_DIR

In [ ]:
from IPython.display import FileLink
#%cd $LESSON_HOME_DIR
#FileLink('data/redux/'+submission_file_name)
FileLink(submission_file_name)

You can download this file and submit on the Kaggle website or use the Kaggle command line tool's "submit" method.

In [ ]:
#kg submit <submission-file> -u <username> -p <password> -c <competition> -m "<message>"
#submitted 20-11-2017 score 0.10853 position ~ 534 out of ~1350 not great result.